# Saving and re-using the betas in the LSMC algorithm

The idea is simple. We use the LSMC with bootstrapping to obtain a better exercise boundary and save the betas. Then, we can use those betas in a seperate LSMC execution.


In [ ]:
# Set the initial variables for the script
import numpy as np
from numpy.linalg import lstsq
import matplotlib.pyplot as plt

# Define parameters
r = 0.06    # Interest rate
K = 1.00    # Strike price
# it is better to work in time-steps than in years
dt = 1/12   # Time-step size in years
nt = 12     # Number of time-steps
T = nt*dt   # Total time to maturity in years

N = 10000      # Number of simulations
M = 9          # polynomial order for cross-sectional regressions
R = 50         # Number of repeats

# For the stock price, here, we will simulate from a Geometric Brownian motion
# We assume a risk-neutral measure
S0 = 1.00   # Initial stock price
sigma = 0.2  # Volatility of the stock


# For reproducibility when doing multiple repeats, we need to get multiple streams of random numbers. I need R streams
SEED = 42  # Seed for reproducibility
SEQ = np.random.SeedSequence(SEED)
SeedSeq = SEQ.spawn(R)
My_R_Streams = [np.random.default_rng(s) for s in SeedSeq]
# Store RNG states before simulation
rng_states = [rng.bit_generator.state for rng in My_R_Streams]
# Because we will loop over the repeats multiple times in the bootstrapping algorithm, 
# we will need to reset the stream each time to ensure we always simulte the same paths
# We don't really need this, though. We could just continue the simulate from the last point.
# However, if we wanted to test convergence as we add more repeats, we would need to reset the RNG state.

# initialize an array to store all average betas for the stopping time approximation
beta_st_average = np.zeros((M+1, nt))

for t_boot in range(nt-1, 0, -1):
    # Restore RNG states to re-simulate from the same point
    for i, rng in enumerate(My_R_Streams):
        rng.bit_generator.state = rng_states[i]
        
    beta_t_boot = np.zeros((M+1, R))  # Store beta coefficients for this bootstrapping iteration

    # loop over all repeats
    for repnum in range(R):
        # We need to clear variables in the loop to avoid carrying over values from previous iterations
        dcf = None
        intrinsic = None
        exec_t = None
        payoff = None
        
        S = np.zeros((N, nt + 1))
        S[:, 0] = np.linspace(0.1, 2, N)
        
        for i in range(1, nt + 1):
            Z = My_R_Streams[repnum].normal(0, 1, N)  # Standard normal random variables
            S[:, i] = S[:, i - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)

        # We will calculate the exercise/intrinsic value for each path at maturity
        # recall that the option to price is a put option
        intrinsic = np.maximum(K - S[:,-1], 0)

        # We create a payoff vector that will contain the discounted exercise value.
        payoff = np.copy(intrinsic)

        # Now we set the vector exec_t to maturity for each path
        exec_t = nt * np.ones((N,), dtype=int)  # All paths would optimally be exercised at time-step 3 (so far))

        # We will do one more thing, for illustration purposes, we will keep the beta coefficients for each time-step in a matrix
        betas = np.zeros((3, nt))  # Store beta coefficients for each time-step

        for t_now in range(nt-1, 0, -1):

            # STEP 1
            dcf = np.exp(-r * dt) * payoff  # Discounted cash flow to time t

            # STEP 2
            itm_paths = np.where(S[:, t_now] < K)[0]  # Identify ITM paths

            # STEP 3
            # Build X matrix for regression (using the stock prices at time t_now)
            # We use a constant, the stock price and the square of the stock price
            x = S[itm_paths, t_now]
            X = np.ones((len(itm_paths), M + 1))
            for j in range(1, M + 1):
                X[:, j] = X[:, j - 1] * x

            # STEP 4
            # Build y vector (the discounted cash flows for ITM paths)
            y = dcf[itm_paths]

            # STEP 5
            # Perform regression to get beta coefficients
            if t_boot<t_now:
                # print(f"using previous beta average")
                # If we are not at the last bootstrapping iteration, we will use the average betas from the previous iteration
                beta = beta_st_average[:, t_now]

            else:
                # print(f"using lstsq to calculate new beta")
                # If we are at the last bootstrapping iteration, we will perform the regression
                # and store the beta coefficients for this time-step
                beta = lstsq(X, y, rcond=None)[0]
                beta_t_boot[:, repnum] = beta
                # now we can exit the loop and continue with the next repeat
                break

            # STEP 6
            # Calculate y_hat to approximate the holding value function
            y_hat = X @ beta

            # Update the payoff for exercised paths to be the current intrinsic value
            intrinsic[itm_paths] = np.maximum(K - S[itm_paths, t_now], 0)

            # STEP 7
            # Update the optimal exercise time for each path
            exec_t[itm_paths] = np.where(y_hat < intrinsic[itm_paths], t_now, exec_t[itm_paths])
            payoff = np.where(exec_t == t_now, intrinsic, dcf)
    
    # We can take the average of the beta coefficients across all repeats
    beta_st_average[:, t_boot] = np.mean(beta_t_boot, axis=1)

np.save('4.4.1_beta_st_average.npy', beta_st_average)    
    